In [25]:
!pip install opendatasets

In [26]:
import opendatasets as od

In [28]:
od.download("https://www.kaggle.com/datasets/zalando-research/fashionmnist")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Dataset URL: https://www.kaggle.com/datasets/zalando-research/fashionmnist


100%|██████████| 68.8M/68.8M [00:00<00:00, 944MB/s]

In [29]:
data_dir = './fashionmnist'

In [31]:
print(data_dir)

./fashionmnist


In [33]:
import torch
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torchvision

In [34]:
torch.manual_seed(42)

In [35]:
transform = T.Compose(
    [T.ToTensor(),
     T.Normalize([0.5],[0.5])]
)

In [39]:
class NeuralNet(nn.Module):
    def __init__(self, input_dims, hidden_dims, out_dims):
        super().__init__()
        self.fc1 = nn.Linear(input_dims, hidden_dims)
        self.fc2 = nn.Linear(hidden_dims, hidden_dims)
        self.fc3 = nn.Linear(hidden_dims, hidden_dims)
        self.fc4 = nn.Linear(hidden_dims, out_dims)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))

        return x

In [40]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [41]:
model = NeuralNet(input_dims = 28 * 28, hidden_dims = 128, out_dims = 10).to(device)

In [ ]:
class MyDataset(Dataset):
    def __init__(self, x, y):
        self.x = x.values
        self.y = y.values

    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return torch.tensor(self.x[idx], dtype = torch.float32), torch.tensor(self.y[idx], dtype = torch.float32)

In [59]:
train_set_dir = "C:\\Users\\harry\\OneDrive\\Desktop\\Generative AI\\fashionmnist\\fashion-mnist_train.csv"

In [60]:
import pandas as pd

In [61]:
data = pd.read_csv(train_set_dir)

In [62]:
print(data.head())

   label  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  \
0      2       0       0       0       0       0       0       0       0   
1      9       0       0       0       0       0       0       0       0   
2      6       0       0       0       0       0       0       0       5   
3      0       0       0       0       1       2       0       0       0   
4      3       0       0       0       0       0       0       0       0   

   pixel9  ...  pixel775  pixel776  pixel777  pixel778  pixel779  pixel780  \
0       0  ...         0         0         0         0         0         0   
1       0  ...         0         0         0         0         0         0   
2       0  ...         0         0         0        30        43         0   
3       0  ...         3         0         0         0         0         1   
4       0  ...         0         0         0         0         0         0   

   pixel781  pixel782  pixel783  pixel784  
0         0         0         

In [63]:
X = data.iloc[:,1:]

In [64]:
print(X.head())

   pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  pixel9  \
0       0       0       0       0       0       0       0       0       0   
1       0       0       0       0       0       0       0       0       0   
2       0       0       0       0       0       0       0       5       0   
3       0       0       0       1       2       0       0       0       0   
4       0       0       0       0       0       0       0       0       0   

   pixel10  ...  pixel775  pixel776  pixel777  pixel778  pixel779  pixel780  \
0        0  ...         0         0         0         0         0         0   
1        0  ...         0         0         0         0         0         0   
2        0  ...         0         0         0        30        43         0   
3        0  ...         3         0         0         0         0         1   
4        0  ...         0         0         0         0         0         0   

   pixel781  pixel782  pixel783  pixel784  
0         0       

In [65]:
y = data.iloc[:,0:1]

In [66]:
from sklearn.model_selection import train_test_split

In [67]:
X_train,X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2)

In [74]:
print(type(X_train))

<class 'pandas.core.frame.DataFrame'>


In [78]:
print(X_train.values[0])

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   1   0   0   4   0   0   0  41
  15   0   0   5   1   1   0   1   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   6   0   0 145 178 142 151 169  56   0   0   0   2   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   2   0   0 154  87
   0   0   0   6 146 105   0   0   1   0   0   0   0   0   0   0   0   0
   0   0   1   0   1   0   0 140 186   0   0   3   0   0 121 209  94   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   4   0  18 215
 113   0   0   2   2   0  55 199 170   0   0   2   

In [68]:
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

(48000, 784)
(12000, 784)
(48000, 1)
(12000, 1)


In [79]:
train_data = MyDataset(X_train, y_train)
val_data = MyDataset(X_val, y_val)

In [80]:
train_loader = DataLoader(train_data, batch_size = 64, shuffle = True, pin_memory = True)
val_loader = DataLoader(val_data, batch_size = 128, shuffle = False)

In [82]:
class Model_training_testing():
    def __init__(self, model, train_loader, val_loader, learning_rate, num_epochs):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr = learning_rate)
        self.criterion = nn.CrossEntropyLoss()
        self.num_epochs = num_epochs
        self.patience = 10
        self.step = 0

    def train(self):
        for i in range(self.num_epochs):
            epoch_loss = 0
            loss = []
            for idx, (x, y) in enumerate(self.train_loader):
                x = x.to(device)
                y = y.to(device)

                pred = self.model(x)

                loss = self.criterion(pred, y)

                self.optimizer.zero_grad()

                loss.backward()

                self.optimizer.step()

                epoch_loss += loss

            loss.append(epoch_loss.detach())

            val_epoch_loss = 0
            val_loss = []

            for idx, (x, y) in enumerate(self.val_loader):
                x = x.to(device)
                y = y.to(device)

                with torch.no_grad():
                    pred = self.model(x)

                loss = self.criterion(pred, y)

                val_epoch_loss += loss

            if val_epoch_loss < self.min_loss:
                self.min_loss = val_epoch_loss
                self.step = 0

            elif val_epoch_loss >= self.min_loss:
                self.step += 1

            if self.step == self.patience:
                return 
            
project = Model_training_testing(model, train_loader, val_loader, 0.001, 5000)

In [83]:
project.model.eval()

test_data = pd.read_csv("C:\\Users\\harry\\OneDrive\\Desktop\\Generative AI\\fashionmnist\\fashion-mnist_test.csv")

In [84]:
X_test = test_data.iloc[:,1:]
y_test = test_data.iloc[:,0:1]

In [85]:
test_dataset = MyDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size = 128, shuffle = False)

In [ ]:
for idx, (x, y) in enumerate(test_loader):
    x = x.to(device)
    y = y.to(device)

    pred = project.model(x)

    pred_label = torch.argmax(pred, dim = 1)

    correct = (pred_label == y)
